In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import pytorch_lightning as pl

from ssl_methods.encoder import ResNet18Encoder
from ssl_methods.decoder import ResNet18Decoder
from ssl_methods.data_modules import ReconstructionDataModule
from ssl_methods.train_modules import BasePretrainingModule

In [2]:
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

In [3]:
model = nn.Sequential(
    ResNet18Encoder(),
    ResNet18Decoder())

In [4]:
data_module = ReconstructionDataModule(preprocess, "../data")
data_module.setup()
training_module = BasePretrainingModule(model)

In [5]:
trainer = pl.Trainer(max_epochs=50)
trainer.fit(training_module, data_module)

/home/lukasz/miniconda3/envs/pssl/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/lukasz/miniconda3/envs/pssl/lib/python3.11/sit ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | model   | Se

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/lukasz/miniconda3/envs/pssl/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/lukasz/miniconda3/envs/pssl/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 1:  75%|███████▌  | 1883/2500 [01:43<00:33, 18.17it/s, v_num=13, train_loss_step=0.0116, val_loss=0.0165, train_loss_epoch=0.0287]

/home/lukasz/miniconda3/envs/pssl/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
